<a href="https://colab.research.google.com/github/DeaAnalytics/univ-workshop/blob/main/notebooks/Python/Sesion6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import plotly
import plotly.express as px
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
pd.set_option('plotting.backend','plotly')

In [3]:
#Leer un archivo .csv, en este caso agregarmos la liga de github que contiene esta tabla y renombrarla como kdf
#Con la función "head" puedo dar un vistazo a los primeros datos de mi tabla y sus encabezados
df = pd.read_csv("https://github.com/DeaAnalytics/univ-workshop/raw/main/data/K_Means_U.csv")
df.head()

,Temperatura,Velocidad,Elongacion,Diametro
0,500,1600,3.4,0.4
1,520,1500,3.5,0.2
2,520,1400,3.4,0.2
3,470,1600,3.2,0.2
4,480,1600,3.1,0.2


In [4]:
#Como podemos ver la magnitud de los valores entre cada variable difieren significativamente, 
#Por lo que debemos escalarlas antes de aplicar clustering
scaler = StandardScaler()
df_scale = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scale.head()

,Temperatura,Velocidad,Elongacion,Diametro
0,-1.021849,-1.227541,0.800654,-1.050031
1,-0.779513,-1.284407,1.032057,-1.312977
2,-0.779513,-1.341272,0.800654,-1.312977
3,-1.385353,-1.227541,0.337848,-1.312977
4,-1.264185,-1.227541,0.106445,-1.312977


In [5]:
# Funcion para calcular la inercia entre los grupos
# la inercia es la suma de las distancias al cuadrado de cada objeto del Cluster a su centroide
def Elbow(df, min_clusters=1, max_clusters=50):
  wcss = []
  n_clusters = []
  for k in range(min_clusters, max_clusters + 1):
    kmeans = KMeans(n_clusters = k, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(df)
    n_clusters.append(k)
    wcss.append(kmeans.inertia_)
  clusters = pd.DataFrame(data=[n_clusters, wcss]).T
  clusters.columns = ["n_clusters", "wcss"]
  clusters.set_index("n_clusters", inplace=True)
  return clusters

In [6]:
clusters = Elbow(df_scale, max_clusters=15)
clusters.plot()

In [7]:
#Ahora iniciemos con el Modelo Kmeans para k=3 (3 cluster):
# El parametro de random_state es para establecer una semilla y el resultado no sea aleatorio en cada ejecucion
k=3
kMeans = KMeans(n_clusters=k, random_state=123)
kMeans.fit(X=df_scale)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=123, tol=0.0001, verbose=0)

In [8]:
kMeans.inertia_

140.965816630747

In [9]:
kMeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0,
       0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2,
       2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [10]:
kMeans.cluster_centers_

array([[-0.05021989,  0.34753171, -0.88029181,  0.28206327],
       [-1.01457897, -1.30487835,  0.84230679, -1.25512862],
       [ 1.13597027,  0.996271  ,  0.09659843,  1.01717187]])

In [11]:
# color continuo NO RECOMENDADO para visualizacion de clusters
fig = px.scatter_matrix(df_scale, dimensions=df_scale.columns, color=kMeans.labels_)
fig.show()

In [12]:
# color discreto RECOMENDADO para visualizacion de clusters
fig = px.scatter_matrix(df_scale, dimensions=df_scale.columns, color=kMeans.labels_.astype(str))
fig.show()